In [1]:
from data_processing.clinvar import getClinvar
from data_processing.main import get_gene_info, read_scoreset
from data_processing.gnomad import queryGnomAD
from data_processing.splice_ai import querySpliceAI
from pathlib import Path
import os
from tqdm.autonotebook import tqdm
import json
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
pd.set_option('display.max_columns', None)
cache_dir = Path("/data/dzeiberg/mave_calibration/cache")
cache_dir.mkdir(exist_ok=True, parents=True)
data_directory = Path("/data/dzeiberg/mave_calibration/data")
dataset_names = [os.path.basename(f.parent) for f in data_directory.glob("**/observations.pkl")]
metadata = []
for dataset in dataset_names:
    with open(data_directory / dataset / "metadata.json") as f:
        metadata.append(dict(**json.load(f), dataset=dataset))
metadata = pd.DataFrame(metadata).set_index("dataset")
gene_info = {}
for uniprot_acc in tqdm(metadata["uniprot_acc"].unique()):
    gene_info[uniprot_acc] = get_gene_info(uniprot_acc, cache_dir="/data/dzeiberg/mave_calibration/cache")
gene_info = pd.DataFrame(gene_info).T

  0%|          | 0/11 [00:00<?, ?it/s]

In [2]:
with open("data_processing/config.json") as f:
    external_config = json.load(f)

read gnomAD missense variants in chromosomal regions of the transcripts for which I have MAVE data

In [3]:
gnomad_records = []
spliceAI_scores = []
for uniprot_acc, r in tqdm(gene_info.iterrows()):
    if (cache_dir / f"gnomad_matches_{r.HGNC_ID}_CHR{r.CHROM}_{r.START}_{r.STOP}.tsv").exists():
        gnomad_record = pd.read_csv(cache_dir / f"gnomad_matches_{r.HGNC_ID}_CHR{r.CHROM}_{r.START}_{r.STOP}.tsv", sep="\t")
    else:
        print(cache_dir / f"gnomad_matches_{r.HGNC_ID}_CHR{r.CHROM}_{r.START}_{r.STOP}.tsv")
        break
        gnomad_record = queryGnomAD(r.CHROM, r.START,r.STOP,r.HGNC_ID,
                                            **external_config["gnomad"],
                                            **external_config['external_tools'],
                                            write_dir=cache_dir)
    gnomad_record = gnomad_record.assign(CHROM=gnomad_record.CHROM.astype(str).str.replace('chr',''),
                            POS=gnomad_record.POS.astype(str),
                            REF=gnomad_record.REF.astype(str),
                            ALT=gnomad_record.ALT.astype(str))
    # read spliceAI scores
    gnomad_records.append(gnomad_record)
    hg38_fp = cache_dir / f"splice_ai_hg38_match_CHR{r.CHROM}_{r.START}_{r.STOP}.tsv"
    if hg38_fp.exists():
        region_spliceAI_scores_hg38 = pd.read_csv(hg38_fp, sep="\t")
    else:
        region_spliceAI_scores_hg38 = querySpliceAI(r['CHROM'],r['START'],r['STOP'],spliceAIFilePath=external_config['splice_ai']['spliceAIFilePath_hg38'],assembly='hg38',write_dir=cache_dir)
    region_spliceAI_scores_hg38 = region_spliceAI_scores_hg38.assign(CHROM=region_spliceAI_scores_hg38.CHROM.astype(str),
                                                           POS=region_spliceAI_scores_hg38.POS.astype(str),
                                                           REF=region_spliceAI_scores_hg38.REF.astype(str),
                                                              ALT=region_spliceAI_scores_hg38.ALT.astype(str),
                                                              Assembly="GRCh38")
    spliceAI_scores.append(region_spliceAI_scores_hg38)
# aggregate gnomAD data    
gnomad_df = pd.concat(gnomad_records)
gnomad_df = gnomad_df.assign(Feature_base=gnomad_df.Feature.str.split(".").str[0])
# aggregate spliceAI scores
spliceAI_df = pd.concat(spliceAI_scores).set_index(['CHROM','POS','REF','ALT'],)
spliceAI_scoreDict = spliceAI_df.spliceAI_score.to_dict()
gnomad_count = gnomad_df.shape[0]
gnomad_df = gnomad_df.set_index(["CHROM","POS","REF","ALT"]).assign(spliceAI_scores=spliceAI_scoreDict)
assert gnomad_count == gnomad_df.shape[0]
# Read ClinVar data
clinvar_df = getClinvar(cache_dir=cache_dir, use_cached_processed_file=True)
clinvar_df = clinvar_df.assign(transcript_base=clinvar_df.transcript.str.split(".").str[0])
# only variants with a base transcript in gene_info (only relevant variants)
clinvar_df[clinvar_df.transcript_base.isin(set(gene_info.MANE_RefSeq_nuc.str.split(".").str[0].values))]
# only GRCh38 Assembly variants
clinvar_df = clinvar_df[clinvar_df.Assembly == "GRCh38"]
# annotate with spliceAI scores

clinvar_df = clinvar_df.assign(Chromosome=clinvar_df.Chromosome.astype(str),
                                 PositionVCF=clinvar_df.PositionVCF.astype(str))
clinvar_count = clinvar_df.shape[0]
clinvar_df = clinvar_df.set_index(["Chromosome","PositionVCF","ReferenceAlleleVCF","AlternateAlleleVCF"]).assign(spliceAI_score=spliceAI_scoreDict)
assert clinvar_count == clinvar_df.shape[0]


0it [00:00, ?it/s]

Using GATK jar /data/utilities/bio/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /data/utilities/bio/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar SelectVariants -V /data/dbs/spliceAI/genome_scores_v1.3_ds.20a701bc58ab45b59de2576db79ac8d0/spliceai_scores.raw.snv.hg38.vcf.gz -L 17:43044295-43170245 --output /data/dzeiberg/mave_calibration/cache/splice_ai_query_result.2024-09-12_12:04:27.458327.vcf
12:04:28.982 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/data/utilities/bio/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
12:04:29.013 INFO  SelectVariants - ------------------------------------------------------------
12:04:29.015 INFO  SelectVariants - The Genome Analysis Toolkit (GATK) v4.4.0.0
12:04:29.015 INFO  SelectVariants - For support and docu

In [4]:
joblib.dump(dict(metadata=metadata, gene_info=gene_info, gnomad_df=gnomad_df, spliceAI_df=spliceAI_df, clinvar_df=clinvar_df), "/data/dzeiberg/mave_calibration/cache/mapping_data.pkl")

['/data/dzeiberg/mave_calibration/cache/mapping_data.pkl']

counts of gnomAD variants for each gene of interest

In [11]:
gnomad_df[gnomad_df.Feature_base.isin(set(gene_info.MANE_RefSeq_nuc.str.split(".").str[0].values))].SYMBOL.value_counts()

SYMBOL
BRCA2    5103
BRCA1    2538
NPC1     1914
MSH2     1852
CHEK2     859
TP53      605
HMBS      494
VHL       405
TPMT      336
DDX3X     268
PTEN      254
Name: count, dtype: int64

In [12]:
data_directory = Path("/data/dzeiberg/mave_calibration/data")
with open(data_directory / "dataset_configs.json") as f:
    dataset_configs = json.load(f)
scoresets = dict()
for dataset_name in dataset_names:
    if dataset_name not in dataset_configs:
        print(f"skipping {dataset_name}")
        continue
    # read raw score
    scoreset = read_scoreset(data_directory / dataset_name / "scoreset.csv")
    # standardize if necessary
    standardize_sample_name = dataset_configs[dataset_name].get('standardize_to',None)
    if standardize_sample_name is not None:
        print(f'standardizing {dataset_name} to {standardize_sample_name}')
        standardization_sample = joblib.load(data_directory / dataset_name / "observations.pkl")[standardize_sample_name]
        mu = standardization_sample.mean()
        sigma = standardization_sample.std()
        scoreset = scoreset.assign(score= (scoreset.score - mu) / sigma)
    # invert if necessary
    if dataset_configs[dataset_name].get('invert',False):
        print(f'inverting {dataset_name} scores')
        scoreset = scoreset.assign(score=-1 * scoreset.score)
    uniprot_acc = metadata.loc[dataset_name,"uniprot_acc"]
    # add metadata
    scoreset = scoreset.assign(GeneID=int(gene_info.loc[uniprot_acc,'GeneID']),
                                MANE_RefSeq_nuc=gene_info.loc[uniprot_acc,'MANE_RefSeq_nuc'],
                                MANE_RefSeq_nuc_base=gene_info.loc[uniprot_acc,'MANE_RefSeq_nuc'].split('.')[0],
                                HGNC_ID=gene_info.loc[uniprot_acc,'HGNC_ID'],
                                Ensembl_nuc=gene_info.loc[uniprot_acc,'ENSEMBL_nuc'],
                                Ensembl_prot=gene_info.loc[uniprot_acc,'ENSEMBL_prot'],
                                Ensembl_nuc_base=gene_info.loc[uniprot_acc,'ENSEMBL_nuc'].split('.')[0],
                                Ensembl_prot_base=gene_info.loc[uniprot_acc,'ENSEMBL_prot'].split('.')[0])
    scoresets[dataset_name] = scoreset


inverting Boettcher_TP53 scores
skipping Erwood_NPC1_RPE1
inverting Giacomelli_p53_nullNutlin3 scores
inverting Giacomelli_p53_wtNutlin3 scores
inverting Jia_MSH2_SSM scores
standardizing Kato_TP53_AIP1nWT to b_lb
standardizing Kato_TP53_BAXnWT to b_lb
standardizing Kato_TP53_GADD45nWT to b_lb
standardizing Kato_TP53_MDM2nWT to b_lb
standardizing Kato_TP53_NOXAnWT to b_lb
standardizing Kato_TP53_P53R2nWT to b_lb
standardizing Kato_TP53_WAF1nWT to b_lb
standardizing Kato_TP53_h1433snWT to b_lb
inverting Kotler_TP53_RelativeFitnessH1299 scores
inverting Kotler_TP53_RelativeFitnessHCT116 scores
skipping Kotler_TP53_inVivoEnrichment
skipping Matreyek_TPMT_VampSeq


In [47]:
def get_gnomAD_subset(ss):
    scoreset_tups_ensembl = set(ss.loc[:,['Ensembl_nuc_base','hgvs_pro']].astype(str).apply(tuple,axis=1).values)
    scoreset_tups_refseq = set(ss.loc[:,['MANE_RefSeq_nuc_base','hgvs_pro']].astype(str).apply(tuple,axis=1).values)
    # get gnomAD variants with available scores
    gnomad_sub = gnomad_df[gnomad_df.loc[:,['Feature_base','hgvs_pro']].astype(str).apply(tuple,axis=1).isin(scoreset_tups_ensembl.union(scoreset_tups_refseq))].reset_index().set_index(['Feature_base','hgvs_pro'])
    scores_ensembl = ss.set_index(['Ensembl_nuc_base','hgvs_pro']).score
    scores_refseq = ss.set_index(['MANE_RefSeq_nuc_base','hgvs_pro']).score
    gnomad_sub['score'] = np.nan
    gnomad_sub['score'].update(scores_ensembl)# = gnomad_sub.assign(score=gnomad_sub.index.map(scores_ensembl)).reset_index()
    gnomad_sub['score'].update(scores_refseq)
    return gnomad_sub.reset_index()

def get_clinvar_subset(ss):
    scoreset_tups = set(ss.loc[:,['MANE_RefSeq_nuc_base','hgvs_pro']].astype(str).apply(tuple,axis=1).values)
    clinvar_sub = clinvar_df[clinvar_df.loc[:,['transcript_base','hgvs_pro']].astype(str).apply(tuple,axis=1).isin(scoreset_tups)].reset_index().set_index(['transcript_base','hgvs_pro'])
    scores = ss.set_index(['MANE_RefSeq_nuc_base','hgvs_pro']).score
    clinvar_sub['score'] = np.nan
    clinvar_sub['score'].update(scores)
    return clinvar_sub.reset_index()

In [50]:
gnomAD_ds = get_gnomAD_subset(scoresets['Findlay_BRCA1_SGE'])
clinvar_ds = get_clinvar_subset(scoresets['Findlay_BRCA1_SGE'])
scoreset = scoresets['Findlay_BRCA1_SGE']

In [49]:
gnomAD_ds[gnomAD_ds.duplicated(subset=["hgvs_pro"],keep=False)].sort_values("hgvs_pro")

,Feature_base,hgvs_pro,CHROM,POS,REF,ALT,ID,QUAL,FILTER,AC,AF,vep,index,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,ALLELE_NUM,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,UNIPROT_ISOFORM,SOURCE,DOMAINS,miRNA,HGVS_OFFSET,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,LoF,LoF_filter,LoF_flags,LoF_info,spliceAI_scores,score
446,ENST00000357654,p.Ala1669Ser,17,43067677,C,A,rs80357087,-10.0,PASS,28,1.841770e-04,A|missense_variant|MODERATE|BRCA1|ENSG00000012...,6068,A,missense_variant,MODERATE,BRCA1,ENSG00000012048,Transcript,ENST00000357654,protein_coding,16/23,NaN,ENST00000357654.9:c.5005G>T,ENSP00000350283.3:p.Ala1669Ser,5118,5005,1669,A/S,Gcc/Tcc,1,NaN,-1,NaN,SNV,HGNC,HGNC:1100,YES,NM_007294.4,NaN,1.0,A2,CCDS11453.1,ENSP00000350283,P38398-1,Ensembl,ENSP_mappings:1jnx&ENSP_mappings:1n5o&ENSP_map...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.401851
447,NM_007294,p.Ala1669Ser,17,43067677,C,A,rs80357087,-10.0,PASS,28,1.841770e-04,A|missense_variant|MODERATE|BRCA1|ENSG00000012...,6068,A,missense_variant,MODERATE,BRCA1,672,Transcript,NM_007294.4,protein_coding,16/23,NaN,NM_007294.4:c.5005G>T,NP_009225.1:p.Ala1669Ser,5118,5005,1669,A/S,Gcc/Tcc,1,NaN,-1,NaN,SNV,EntrezGene,HGNC:1100,YES,ENST00000357654.9,NaN,NaN,NaN,NaN,NP_009225.1,NaN,RefSeq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.401851
448,ENST00000357654,p.Ala1669Ser,17,43067677,C,A,rs80357087,-10.0,PASS,122,8.353400e-05,A|missense_variant|MODERATE|BRCA1|ENSG00000012...,6069,A,missense_variant,MODERATE,BRCA1,ENSG00000012048,Transcript,ENST00000357654,protein_coding,16/23,NaN,ENST00000357654.9:c.5005G>T,ENSP00000350283.3:p.Ala1669Ser,5118,5005,1669,A/S,Gcc/Tcc,1,NaN,-1,NaN,SNV,HGNC,HGNC:1100,YES,NM_007294.4,NaN,1.0,A2,CCDS11453.1,ENSP00000350283,P38398-1,Ensembl,ENSP_mappings:1jnx&ENSP_mappings:1n5o&ENSP_map...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.401851
449,NM_007294,p.Ala1669Ser,17,43067677,C,A,rs80357087,-10.0,PASS,122,8.353400e-05,A|missense_variant|MODERATE|BRCA1|ENSG00000012...,6069,A,missense_variant,MODERATE,BRCA1,672,Transcript,NM_007294.4,protein_coding,16/23,NaN,NM_007294.4:c.5005G>T,NP_009225.1:p.Ala1669Ser,5118,5005,1669,A/S,Gcc/Tcc,1,NaN,-1,NaN,SNV,EntrezGene,HGNC:1100,YES,ENST00000357654.9,NaN,NaN,NaN,NaN,NP_009225.1,NaN,RefSeq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.401851
451,NM_007294,p.Ala1669Thr,17,43067677,C,T,rs80357087,-10.0,PASS,1,6.847040e-07,T|missense_variant|MODERATE|BRCA1|ENSG00000012...,6070,T,missense_variant,MODERATE,BRCA1,672,Transcript,NM_007294.4,protein_coding,16/23,NaN,NM_007294.4:c.5005G>A,NP_009225.1:p.Ala1669Thr,5118,5005,1669,A/T,Gcc/Acc,1,NaN,-1,NaN,SNV,EntrezGene,HGNC:1100,YES,ENST00000357654.9,NaN,NaN,NaN,NaN,NP_009225.1,NaN,RefSeq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,0.033971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,ENST00000357654,p.Val83Ile,17,43104922,C,T,rs1060502343,-10.0,PASS,6,4.105090e-06,T|missense_variant|MODERATE|BRCA1|ENSG00000012...,16841,T,missense_variant,MODERATE,BRCA1,ENSG00000012048,Transcript,ENST00000357654,protein_coding,5/23,NaN,ENST00000357654.9:c.247G>A,ENSP00000350283.3:p.Val83Ile,360,247,83,V/I,Gtt/Att,1,NaN,-1,NaN,SNV,HGNC,HGNC:1100,YES,NM_007294.4,NaN,1.0,A2,CCDS11453.1,ENSP00000350283,P38398-1,Ensembl,ENSP_mappings:1jm7&Gene3D:3&PIRSF:PIRSF001734&...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.07,-0.093432
777,NM_007294,p.Val8Ile,17,43124075,C,T,rs528902306,-10.0,PASS,1,6.843080e-07,T|missense_variant|MODERATE|BRCA1|ENSG00000012...,21127,T,missense_variant,MODERATE,BRCA1,672,Transcript,NM_007

# Case by case identify discrepancies between this pipeline and the process_data pipeline for Findlay BRCA1 SGE

In [53]:
from main import load_data
findlay_data = load_data(data_directory=Path("/data/dzeiberg/mave_calibration_debug_data"),dataset_id="Findlay_BRCA1_SGE")
findlay_observations = joblib.load(Path("/data/dzeiberg/mave_calibration_debug_data") / "Findlay_BRCA1_SGE" / "hgvs_pro.pkl")

In [55]:
set(findlay_observations['p_lp']) - set(clinvar_ds.loc[clinvar_ds.is_pathogenic,'hgvs_pro'].values)

set()

In [56]:
set(findlay_observations['b_lb']) - set(clinvar_ds.loc[clinvar_ds.is_benign,'hgvs_pro'].values)

set()

In [58]:
missing_from_gnomAD = set(findlay_observations['gnomad']) - set(gnomAD_ds.hgvs_pro.values)

In [62]:
gene_info

,seq,MANE_RefSeq_nuc,gene_name,MANE_RefSeq_prot,CHROM,START,STOP,HGNC_ID,STRAND,GeneID,ENSEMBL_nuc,ENSEMBL_prot
P38398,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...,NM_007294.4,BRCA1,NP_009225.1,17,43044295,43170245,HGNC:1100,-,672,ENST00000357654.9,ENSP00000350283.3
P04637,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,NM_000546.6,TP53,NP_000537.3,17,7661779,7687546,HGNC:11998,-,7157,ENST00000269305.9,ENSP00000269305.4
P51587,MPIGSKERPTFFEIFKTRCNKADLGPISLNWFEELSSEAPPYNSEP...,NM_000059.4,BRCA2,NP_000050.3,13,32315086,32400268,HGNC:1101,+,675,ENST00000380152.8,ENSP00000369497.3
O15118,MTARGLALGLLLLLLCPAQVFSQSCVWYGECGIAYGDKRYNCEYSG...,NM_000271.5,NPC1,NP_000262.2,18,23506184,23586506,HGNC:7897,-,4864,ENST00000269228.10,ENSP00000269228.4
P40337,MPRRAENWDEAEVGAEEAGVEEYGPEEDGGEESGAEESGPEESGPE...,NM_000551.4,VHL,NP_000542.1,3,10141778,10153667,HGNC:12687,+,7428,ENST00000256474.3,ENSP00000256474.3
O96017,MSRESDVEAQQSHGSSACSQPHGSVTQSQGSSSQSQGISSSSTSTM...,NM_007194.4,CHEK2,NP_009125.1,22,28687743,28742422,HGNC:16627,-,11200,ENST00000404276.6,ENSP00000385747.1
P43246,MAVQPKETLQLESAAEVGFVRFFQGMPEKPTTTVRLFDRGDFYTAH...,NM_000251.3,MSH2,NP_000242.1,2,47403067,47663146,HGNC:7325,+,4436,ENST00000233146.7,ENSP00000233146.2
P60484,MTAIIKEIVSRNKRRYQEDGFDLDLTYIYPNIIAMGFPAERLEGVY...,NM_000314.8,PTEN,NP_000305.3,10,87862638,87971930,HGNC:9588,+,5728,ENST00000371953.8,ENSP00000361021.3
P51580,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,NM_000367.5,TPMT,NP_000358.1,6,18128311,18155077,HGNC:12014,-,7172,ENST00000309983.5,ENSP00000312304.4
O00571,MSHVAVENALGLDQQFAGLDLNSSDNQSGGSTASKGRYIPPHLRNR...,NM_001356.5,DDX3X,NP_001347.3,X,41333348,41364472,HGNC:2745,+,1654,ENST00000644876.2,ENSP00000494040.1


In [68]:
set(gene_info.loc[metadata.loc['Findlay_BRCA1_SGE','uniprot_acc'],['MANE_RefSeq_nuc','ENSEMBL_nuc']].apply(lambda x: x.split('.')[0]).values)

{'ENST00000357654', 'NM_007294'}

In [72]:
gnomad_df[(gnomad_df.hgvs_pro.isin(missing_from_gnomAD)) & (gnomad_df.HGNC_ID == gene_info.loc[metadata.loc['Findlay_BRCA1_SGE','uniprot_acc'],'HGNC_ID'])].Feature.unique()# & (gnomad_df.Feature_base.isin(set(gene_info.loc[metadata.loc['Findlay_BRCA1_SGE','uniprot_acc'],['MANE_RefSeq_nuc','ENSEMBL_nuc']].apply(lambda x: x.split('.')[0]).values)))]

array(['ENST00000493795', 'NM_007297.4', 'ENST00000591849',
       'ENST00000586385', 'ENST00000471181', 'NM_007300.4',
       'ENST00000461574', 'ENST00000644379', 'ENST00000497488',
       'ENST00000412061', 'ENST00000473961', 'ENST00000484087',
       'ENST00000487825', 'ENST00000493919', 'ENST00000652672',
       'ENST00000477152', 'ENST00000489037', 'ENST00000644555'],
      dtype=object)

In [84]:
def plot_sample(scores,ax=None,label=""):
    if ax is None:
        fig,ax = plt.subplots()
    sns.histplot(scores,label=f"{label} (n={len(scores)})",ax=ax,stat='density',bins=15)
    ax.legend()

In [ ]:
for dataset_name, scoreset in tqdm(scoresets.items(),total=len(scoresets)):
    gnomAD_ds = get_gnomAD_subset(scoresets[dataset_name])
    clinvar_ds = get_clinvar_subset(scoresets[dataset_name])
    savedir = data_directory / dataset_name / "processed"
    savedir.mkdir(exist_ok=True,parents=True)
    gnomAD_ds.to_csv(savedir / "gnomAD.csv")
    clinvar_ds.to_csv(savedir / "ClinVar.csv")
    fig,ax = plt.subplots(4,1,figsize=(10,12),sharex=True,sharey=True)
    plot_sample(clinvar_ds[(clinvar_ds.is_pathogenic) & \
                    (clinvar_ds.snv_category == "missense") & \
                        (clinvar_ds.spliceAI_score < .5)].drop_duplicates('Name').score.values,ax=ax[0],label="P/LP")

    plot_sample(clinvar_ds[(clinvar_ds.is_benign) & \
                    (clinvar_ds.snv_category == "missense") & \
                        (clinvar_ds.spliceAI_score < .5)].drop_duplicates('Name').score.values,ax=ax[1],label="B/LB")
    plot_sample(gnomAD_ds.score,ax=ax[2],label="gnomAD")
    clinvar_ds = clinvar_ds.reset_index()
    silent_pathogenic_variants = set(clinvar_ds.loc[(clinvar_ds.snv_category == "silent") & \
                                                    (clinvar_ds.is_pathogenic),'hgvs_pro'].values)
    plot_sample(scoreset[(scoreset.synonymous) & \
                            (~scoreset.hgvs_pro.isin(silent_pathogenic_variants))].score,ax=ax[3],label="Synonymous")
    fig.savefig(savedir / "score_distributions.jpeg",dpi=300,bbox_inches='tight')
    plt.close(fig)

In [ ]:
gnomad_df.loc[(gnomad_df.HGNC_ID == gene_info.loc['P38398','HGNC_ID']) & \
                (gnomad_df.hgvs_pro.isin(set(scoresets['Findlay_BRCA1_SGE'].hgvs_pro.values))) &\
                    ((gnomad_df.spliceAI_scores.isna()) | (gnomad_df.spliceAI_scores < .5)) & \
                        (~gnomad_df.hgvs_pro.isin(set(gnomAD_ds.hgvs_pro.values)))]

In [ ]:
get_clinvar_subset??

In [ ]:
clinvar_df.loc[(clinvar_df.transcript == gene_info.loc['P38398','MANE_RefSeq_nuc']) & \
                (clinvar_df.hgvs_pro.isin(set(scoresets['Findlay_BRCA1_SGE'].hgvs_pro.values))) &\
                (clinvar_df.is_pathogenic) & \
                    (clinvar_df.snv_category == "missense") & \
                    ((clinvar_df.spliceAI_score.isna()) | (clinvar_df.spliceAI_score < .5))]# & \
                        #(~clinvar_df.hgvs_pro.isin(set(clinvar_ds.reset_index().loc[clinvar_ds.is_pathogenic].hgvs_pro.values)))]

In [ ]:
len(set(findlay_observations['b_lb']))

In [ ]:
clinvar_df.loc[(clinvar_df.hgvs_pro.isin(set(findlay_observations['b_lb']))) & \
                (clinvar_df.transcript == gene_info.loc['P38398','MANE_RefSeq_nuc']) & \
                    ((clinvar_df.spliceAI_score.isna()) | (clinvar_df.spliceAI_score < .5)) & \
                        (clinvar_df.hgvs_pro.isin(set(scoresets['Findlay_BRCA1_SGE'].hgvs_pro.values))) & \
                    (~clinvar_df.hgvs_pro.isin(set(clinvar_ds.loc[(clinvar_ds.is_benign) & \
                                                                    (clinvar_ds.snv_category == "missense") & \
                                                                        ((clinvar_ds.spliceAI_score < .5) | \
                                                                            (clinvar_ds.spliceAI_score.isna()))].hgvs_pro.values)))].drop_duplicates('hgvs_pro')

In [ ]:
clinvar_ds.loc[(clinvar_ds.is_pathogenic) & \
                (clinvar_ds.snv_category == "missense") & \
                    ((clinvar_ds.spliceAI_score < .5) | \
                        (clinvar_ds.spliceAI_score.isna()))].drop_duplicates('hgvs_pro').shape

In [ ]:
clinvar_ds.loc[(clinvar_ds.is_benign) & \
                (clinvar_ds.snv_category == "missense") & \
                    ((clinvar_ds.spliceAI_score < .5) | \
                        (clinvar_ds.spliceAI_score.isna()))].drop_duplicates('hgvs_pro').shape

In [16]:
results_dir = Path("/data/dzeiberg/mave_calibration/results_08_26_24/figs3")
scoreset_thresholds = {}
for fp in results_dir.glob("*.json"):
    with open(fp) as f:
        summary = json.load(f)
    scoreset_thresholds[os.path.basename(fp).replace(".json","")] = dict(p_lp=summary['pathogenic_score_thresholds'],b_lb=summary['benign_score_thresholds'])

In [17]:
def assign_assay_evidence_strength(score, thresholds):
    if np.isnan(score):
        return 0
    for threshold,points in list(zip(thresholds["p_lp"],[1,2,3,4,8]))[::-1]:
        if np.isnan(threshold):
            continue
        if score <= threshold:
            return points
    for threshold,points in list(zip(thresholds["b_lb"],[-1,-2,-3,-4,-8]))[::-1]:
        if np.isnan(threshold):
            continue
        if score >= threshold:
            return points
    return 0

In [ ]:
scoreset[scoreset.hgvs_pro == "p.Ala1823Pro"]

In [260]:
clinvar_scored = pd.merge(clinvar_df,scoreset_df,
            left_on=['hgvs_pro','transcript'],
            right_on=['hgvs_pro','MANE_RefSeq_nuc'],
            how='left')
gnomad_scored = pd.merge(gnomad_df,scoreset_df,
            left_on=['hgvs_pro','HGNC_ID'],
            right_on=['hgvs_pro','HGNC_ID'],
            how='left')

In [ ]:
clinvar_scored[(clinvar_scored.hgvs_pro == "p.Ala1823Pro")]

In [ ]:
f"{clinvar_scored.shape[0]} ClinVar score records (1 record per score)" 

In [ ]:
f"{gnomad_scored.shape[0]} gnomAD score records (1 record per score)"

In [267]:
# assay_cols = [*[f"score_{k}" for k in scoreset_thresholds.keys()],*[f"strength_{k}" for k in scoreset_thresholds.keys()]]
assay_cols = ['score_Findlay_BRCA1_SGE','strength_Findlay_BRCA1_SGE']

In [268]:
clinvar_assays_aggregated = pd.merge(clinvar_scored.loc[:,list(set(clinvar_scored.columns) - set(assay_cols))].drop_duplicates(subset=['hgvs_pro','transcript']),
                                    clinvar_scored.loc[:,['hgvs_pro','transcript',*assay_cols]].groupby(['hgvs_pro','transcript'], as_index=True).agg(np.nanmax),
                                    on=['hgvs_pro','transcript'],)


In [269]:
gnomad_assays_aggregated = pd.merge(gnomad_scored.loc[:,list(set(gnomad_scored.columns) - set(assay_cols))].drop_duplicates(subset=['hgvs_pro','HGNC_ID']),
                                    gnomad_scored.loc[:,['hgvs_pro','HGNC_ID',*assay_cols]].groupby(['hgvs_pro','HGNC_ID'], as_index=True).agg(np.nanmax),
                                    on=['hgvs_pro','HGNC_ID'],)

In [270]:
gnomad_assays_aggregated = gnomad_assays_aggregated.assign(CHROM=gnomad_assays_aggregated.CHROM.str.replace('chr',''))

In [271]:
splice_altering_variants = pd.concat([pd.read_csv(f) for f in data_directory.glob("**/*spliceAI_filtered.csv")])
splice_altering_variants = splice_altering_variants.assign(POS=splice_altering_variants.POS.astype(str),
                                                            CHROM=splice_altering_variants.CHROM.astype(str))

In [272]:
clinvar_assays_aggregated_wo_splice = clinvar_assays_aggregated[~clinvar_assays_aggregated.Name.isin(set(splice_altering_variants.Name.values))]

In [ ]:
clinvar_assays_aggregated.shape[0], clinvar_assays_aggregated_wo_splice.shape[0]

In [274]:
splice_set = set(splice_altering_variants.loc[:,["CHROM","POS","REF","ALT"]].astype(str).apply(tuple,axis=1).values)
gnomad_assays_aggregated_wo_splice = gnomad_assays_aggregated[gnomad_assays_aggregated.loc[:,['CHROM','POS','REF','ALT']].astype(str).apply(tuple, axis=1).apply(lambda x: x not in splice_set)]

In [ ]:
gnomad_assays_aggregated.shape[0], gnomad_assays_aggregated_wo_splice.shape[0]

In [ ]:
color_palette = sns.choose_diverging_palette()

In [ ]:
# strength_col = "strength_Findlay_BRCA1_SGE"
Normalize_counts = True
strength_range = np.array([-8,-4,-3,-2,-1,0,1,2,3,4,8]).astype(float)
color = {-8:color_palette[0],-4:color_palette[1],-3:color_palette[2],-2:color_palette[3],-1:color_palette[4],
        0:color_palette[5],1:color_palette[6],2:color_palette[7],3:color_palette[8],4:color_palette[9],8:color_palette[10]}
# for strength_col in df_scoreset.columns[df_scoreset.columns.str.startswith("strength_")]:    
# for dataset_name in scoreset_thresholds.keys():
for dataset_name in ["Findlay_BRCA1_SGE",]:
    masks = {"P/LP":(~clinvar_assays_aggregated_wo_splice.loc[:,f"strength_{dataset_name}"].isna()) & clinvar_assays_aggregated_wo_splice.is_pathogenic,
            "B/LB":(~clinvar_assays_aggregated_wo_splice.loc[:,f"strength_{dataset_name}"].isna()) & clinvar_assays_aggregated_wo_splice.is_benign,
            "VUS":(~clinvar_assays_aggregated_wo_splice.loc[:,f"strength_{dataset_name}"].isna()) & (clinvar_assays_aggregated_wo_splice.ClinicalSignificance == "Uncertain significance"),
            "Conflicting":(~clinvar_assays_aggregated_wo_splice.loc[:,f"strength_{dataset_name}"].isna()) & (clinvar_assays_aggregated_wo_splice.ClinicalSignificance == "Conflicting classifications of pathogenicity"),
            "Missense":(~clinvar_assays_aggregated_wo_splice.loc[:,f"strength_{dataset_name}"].isna()) & (clinvar_assays_aggregated_wo_splice.snv_category == "missense")}
            
    strengths = {f"{k} (n={mask.sum():,d})": clinvar_assays_aggregated_wo_splice[mask].loc[:,f"strength_{dataset_name}"].value_counts(normalize=Normalize_counts).fillna(0).astype(float) for k,mask in masks.items()}
    gnomAD_mask = ~gnomad_assays_aggregated.loc[:,f"strength_{dataset_name}"].isna()
    strengths[f"gnomAD (n={gnomAD_mask.sum():,d})"] = gnomad_assays_aggregated.loc[gnomAD_mask,f"strength_{dataset_name}"].value_counts(normalize=Normalize_counts).fillna(0).astype(float)
    strengths = pd.DataFrame.from_dict(strengths,orient='index')
    strengths = strengths.rename(columns={s : int(s) for s in strengths.columns})
    for s in strength_range:
        if s not in strengths.columns:
            strengths[s] = 0
    strengths = strengths.loc[:,strengths.columns.sort_values()]
    strengths.plot(kind='barh',
                    stacked=True,
                    color=color)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    fig = plt.gcf()
    plt.title(f"{dataset_name.replace('_',' ')} strength distribution")
    # fig.savefig(results_dir / f"{dataset_name}_evidence_strengths.jpg",bbox_inches='tight',dpi=300)
    plt.show()
    plt.close()


In [279]:
expected_hgvs_pro = joblib.load(data_directory / dataset_name / "hgvs_pro.pkl")

In [ ]:
len(expected_hgvs_pro['p_lp'])

In [ ]:
masks['P/LP'].sum()

In [ ]:
clinvar_assays_aggregated_wo_splice.loc[(masks['B/LB']) & (~clinvar_assays_aggregated_wo_splice.hgvs_pro.isin(set(expected_hgvs_pro['b_lb'].values))),'hgvs_pro']

In [ ]:
set(expected_hgvs_pro['b_lb'].values) - set(clinvar_assays_aggregated_wo_splice.loc[masks['B/LB'],'hgvs_pro'].values)

In [ ]:
clinvar[(clinvar.hgvs_pro == "p.Thr37=") & (clinvar.GeneSymbol == "BRCA1")]

In [ ]:
scoreset[scoreset.hgvs_pro == "p.Thr37="]

In [ ]:
clinvar_scored[(clinvar_scored.hgvs_pro == "p.Thr37=")]

In [ ]:
clinvar_assays_aggregated[clinvar_assays_aggregated.hgvs_pro == "p.Thr37="]

In [ ]:
set(expected_hgvs_pro['p_lp'].values) - set(clinvar_assays_aggregated_wo_splice.loc[masks['P/LP'],'hgvs_pro'].values)

In [ ]:
splice_altering_variants[splice_altering_variants.Name == "NM_007294.4(BRCA1):c.5467G>C (p.Ala1823Pro)"]